# DNN estimator

In [1]:
# filter out future warnings from numpy on tensorflow calls
import warnings  
import numpy as np

with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    import tensorflow as tf
    from tensorflow.compat.v1.estimator import DNNClassifier

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Dataset loading, splitting, standardization

In [2]:
dataset = fetch_openml('iris', version=1)
train_data, test_data, train_target, test_target = train_test_split(dataset.data, dataset.target)

In [3]:
n_classes = len(set(dataset.target))

In [4]:
# standardization trained on train, reused on test dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data)
X_test = scaler.transform(test_data)

In [5]:
# string to class index
class_index_map = { k: v for v, k in enumerate(sorted(list(set(dataset.target))))}
index_class_map = { v: k for k,v in class_index_map.items() }

class_index_mapping = np.vectorize(lambda x: class_index_map[x])

In [6]:
y_train = class_index_mapping(train_target).reshape(-1,1)
y_test = class_index_mapping(test_target).reshape(-1,1)

## MLP Training

In [7]:
feature_columns = [ tf.feature_column.numeric_column(key=feature_name) for feature_name in dataset.feature_names ]

estimator = DNNClassifier(feature_columns=feature_columns,
                          hidden_units = [300,100],
                          n_classes = n_classes)

batch_size = 50

train_spec = tf.estimator.TrainSpec(input_fn=tf.estimator.inputs.numpy_input_fn(x = {'sepallength' : X_train[:,0],
                                                                                     'sepalwidth' : X_train[:,1],
                                                                                     'petallength' : X_train[:,2],
                                                                                     'petalwidth' : X_train[:,3]
                                                                                    },
                                                                                y = y_train, 
                                                                                batch_size=batch_size, 
                                                                                num_epochs=None,
                                                                                shuffle=True),
                                    max_steps=1000)
                                    
eval_spec = tf.estimator.EvalSpec(input_fn=tf.estimator.inputs.numpy_input_fn(x = {'sepallength' : X_test[:,0],
                                                                                   'sepalwidth' : X_test[:,1],
                                                                                   'petallength' : X_test[:,2],
                                                                                   'petalwidth' : X_test[:,3]
                                                                                  },
                                                                              y = y_test, 
                                                                              num_epochs=None,
                                                                              shuffle=False))

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_model_dir': '/tmp/tmpzy0fcz3i', '_num_ps_replicas': 0, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_protocol': None, '_train_distribute': None, '_save_checkpoints_secs': 600, '_service': None, '_master': '', '_task_type': 'worker', '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_eval_distribute': None, '_evaluation_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f95a64250f0>, '_is_chief': True, '_device_fn': None}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluatio

({'accuracy': 0.94734377,
  'average_loss': 0.29450712,
  'global_step': 1000,
  'loss': 37.69691},
 [])